In [ ]:
#comment out if using a local environment
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd "gdrive/My Drive/TopicModelling2/TopicModelling"

/content/gdrive/My Drive/TopicModelling2/TopicModelling


In [ ]:
#download spacy's french language model and pyLDAvis for Topic Modelling visualization
!python -m spacy download fr_core_news_sm
!pip install pyLDAvis

In [ ]:
import pandas as pd
import numpy as np
import warnings
from collections import Counter

#from utilsTM import corpusVisualizer, Output, deTokenize, compareCorpuses #Output is from ipywidgets
from utilsTM import *

from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases
from gensim.models.ldamodel import LdaModel, CoherenceModel
from gensim.corpora.dictionary import Dictionary

import pyLDAvis
import pyLDAvis.gensim #this is needed

import spacy

#import nltk

### Aim Of The Project
The aim of this notebook is to perform topic modelling on a corpus of company descriptions extracted from the [Commercial Register of Geneva](https://www.ge.ch/recherche-entreprises-registre-du-commerce-geneve).  
This could be used to analyze the economic landscape of the city and to build similarities from the topics embeddings.  
*(Please refer to the commercial register of Geneva for information on restrictions of use of the data)*

### Preprocessing the corpus
To get meaningful topics we need to remove words which are not relevant to categorize company descriptions.  
The steps are the following (the order can be changed):


1.   Remove punctations, lower case, accents ...
2.   Remove links, frequent unuseful information
3.   Tokenize
4.   Remove stop words, named entities
5.   Remove words other than noun, adjectif, verb or adverb
6.   Lemmatize
7.   Build bi-grams and tri-grams
8.   Remove frequent words, list of manually chosen words, verbs 

Not that the last step is specific to this task.



In [ ]:
#remove deprecation warning messages from ipywidgets
warnings.filterwarnings(action="ignore")

#TRY LDA FROM PYSPARK MAYBE YOU CAN GET BETTER RESULTS...

Preprocessing step functions

In [ ]:
# a list of processing steps which can be used as in different order in a processing pipeline, these functions are partly reusable.

def removeCfs(corpus):
  '''a lot of doc contain (cf. statuts pour but complet), (cf. but de l'établissement principal), ... at the end of the description. We remove them.'''
  corpus = [doc[:doc.find("(cf")] for doc in corpus]
  return corpus

def gensimPreprocess(corpus):
  '''perform a number of preliminary preprocessing steps using gensim's simple_preprocess() (remove punctations, lowercase etc...).'''
  corpus = [simple_preprocess(doc,deacc=False,max_len=30) for doc in corpus]
  return corpus

def spacyPreprocess(corpus):
  ''''Remove stopwords, punctation, named entities, words that are not a noun, adjectif, verb or adverb, lemmatize.'''
  nlp = spacy.load("fr_core_news_sm") #load a spaCy french language model
  keepPos = ['NOUN', 'ADJ', 'VERB', 'ADV']
  corpusNew = []
  for doc in deTokenize(corpus):
    doc = nlp(doc) #perform text analysis on "doc" with the model from spacy.
    doc = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop and (token.pos_ in keepPos) and not token.ent_type]
    corpusNew.append(doc)
  
  return corpusNew

def removeVerbs(corpus):
  '''Remove french verbs, spacy POS for verbs was not good, so we had to scrape a full list of french verbs (refer to the notebook ScrapingAllFrenchVerbs.ipynb).'''
  with open(file="frenchVerbs.txt",mode="r") as f:
    FRENCHVERBS = list(f.readlines())
    FRENCHVERBS = [verb.replace("\n","").strip() for verb in FRENCHVERBS]
  corpus = [[word for word in doc if word not in FRENCHVERBS] for doc in corpus]
  return corpus

def constructBigrams(corpus):
  '''Extract frequent set of two words.'''
  corpus = list(Phrases(sentences=corpus,min_count=2)[corpus])
  return corpus

def removeAccents(corpus):
  corpus = [simple_preprocess(doc,deacc=True,max_len=30) for doc in deTokenize(corpus)]
  return corpus
  
def removeNonDiscriminative(corpus):
  '''Remove word, which are present in most of the documents and are noise from a domain specific vocabulary (business) rather than discriminative features.'''
  #Take the most frequent word, manually set the threshold for the most common word number and keep some words which might be due to a frequent topic
  cnt = Counter([word for doc in corpus for word in doc])
  keep = ["financier","immobilier","courtage","batiment","materiel","construction","importation_exportation",
        "informatique","bien_immobilier","investissement","industriel","public","prohiber_lfaie","financement"]
  #remove 63 most common
  remove = [k for k,_ in cnt.most_common(100) if k not in keep]
  manual = ["but","pays","genre","aupres","groupe","lien","principalement","commercer","rapport","edition","geneve","premier","tiers"] #a list of unwanted words
  remove = remove + manual

  #remove words
  corpus = [[word for word in doc if word not in remove] for doc in corpus]
  
  return corpus

def removeEmpty(corpus):
  return [doc for doc in corpus if doc]

Preprocessing

In [ ]:
#read the data from a csv file. The data was extracted from the commercial register of Geneva, and consists on company descriptions in French.
corpus0 = pd.read_csv("CommercialRegisterData.csv").Objet.to_list()

#For this first experiment we consider only 10'000 descriptions, but the register contains up to 50'000 companies.
print("Corpus size :",len(corpus0))

In [ ]:
# we will use a small UI to vizualize the corpus more easily, as we go along the different preprocessing steps
Out0 = Output()
corpusVisualizer(corpus0,Out0)

In [ ]:
corpus1 = gensimPreprocess(removeCfs(corpus0))
Out1 = Output()
corpusVisualizer(deTokenize(corpus1),Out1)

In [ ]:
corpus2 = spaCyPreprocess(corpus1)
Out1 = Output()
corpusVisualizer(deTokenize(corpus2),Out1)

In [ ]:
def spacyPre(corpus):
  #3. Remove stopwords, lemmatize
  nlp = spacy.load("fr_core_news_sm")
  corpusNew = []
  for doc2 in deTokenize(corpus):
    doc2 = nlp(doc2)
    doc3 = [token.lemma_ for token in doc2 if not token.is_stop if not token.text in FRENCHVERBS]
    corpusNew.append(doc3)
  
  return corpusNew

In [ ]:
def stemming(corpus):
  #stemming, we try the french stemmer from nltk
  from nltk.stem.snowball import FrenchStemmer
  stemmer = FrenchStemmer()
  return [[stemmer.stem(word) for word in doc] for doc in corpus]

In [ ]:
  with open(file="frenchVerbs.txt",mode="r") as f:
    FRENCHVERBS = list(f.readlines())
    FRENCHVERBS = [verb.replace("\n","").strip() for verb in FRENCHVERBS]

In [ ]:
#steps1 and 25 topics : 0.47

preprocessingSteps1 = [removeCfs, gensimPreprocess, spacyPreprocess, removeAccents, constructBigrams, removeNonDiscriminative, removeEmpty]
preprocessingSteps2 = [removeCfs, gensimPreprocess, removeVerbs,spaCyPreprocess, removeAccents, constructBigrams, removeNonDiscriminative, removeEmpty]
preprocessingSteps3 = [removeCfs, gensimPreprocess,constructBigrams, spacyPreprocess,removeVerbs, removeAccents, removeNonDiscriminative, removeEmpty]

#Probably inefficient for large datasets
corpuses = [corpus0]
for step in preprocessingSteps3:
  corpuses.append(step(corpuses[-1]))

In [ ]:
corpusFinal = corpuses[-1]

### Fitting the model

In [ ]:
id2word = Dictionary(documents=corpusFinal)

In [ ]:
corpusBow = [id2word.doc2bow(doc) for doc in corpusFinal]

In [ ]:
#without removing frequent words, coherence 0.58, 15 topics

In [ ]:
ntopics = 20
rs = 100
model = LdaModel(corpus=corpusBow,num_topics=ntopics,id2word=id2word,chunksize=2000,
                 passes=20,minimum_probability=0.0001,random_state=rs,per_word_topics=False)
coherence = CoherenceModel(model=model,texts=corpusFinal,dictionary=id2word).get_coherence()
print("Average correlation between multi-words in topics",round(coherence,4))

Average correlation between multi-words in topics 0.4443


In [ ]:
#80 not so bad,10 not bad, 15 0.5 !!! #sometimes bigrams are confusing the thign
#model.save(fname="model80topics")

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpusBow, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
6     -0.337633 -0.029074       1        1  8.787130
8      0.020682  0.264861       2        1  6.407988
11    -0.245093 -0.054886       3        1  6.247505
12     0.062076 -0.138229       4        1  6.213078
7     -0.171192 -0.067746       5        1  5.334021
19    -0.031881  0.054462       6        1  5.014225
3      0.098594 -0.056675       7        1  4.984471
13     0.079685 -0.093977       8        1  4.929875
10     0.031896  0.118528       9        1  4.897031
4      0.003792  0.075994      10        1  4.712512
16     0.026470 -0.058600      11        1  4.694821
14     0.086689 -0.108401      12        1  4.560906
2      0.010301  0.199624      13        1  4.531794
18     0.026658  0.038097      14        1  4.460622
5     -0.010965 -0.072048      15        1  4.413465
9      0.094050 -0.037768      16        1  4.398043
0      0.010972  0.105880      17        1  4.192514
17     0.088540  0.009327      18        1  4.035680
1      0.059794 -0.055901      19        1  3.669622
15     0.096566 -0.093468      20        1  3.514697, topic_info=             Term        Freq       Total Category  logprob  loglift
8      immobilier  893.000000  893.000000  Default  30.0000  30.0000
164     financier  703.000000  703.000000  Default  29.0000  29.0000
118  construction  649.000000  649.000000  Default  28.0000  28.0000
151  informatique  404.000000  404.000000  Default  27.0000  27.0000
4        courtage  403.000000  403.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
931        centre   22.091643  129.185359  Topic20  -4.5030   1.5822
807        enfant   20.887358  132.884839  Topic20  -4.5590   1.4979
836     residence   12.132964   15.831020  Topic20  -5.1023   3.0822
63         oeuvre   16.213360   97.314977  Topic20  -4.8124   1.5561
382           art   14.735570  116.126388  Topic20  -4.9079   1.2838

[984 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
2786     15  0.896563     acceptation
653       7  0.753591           acces
653      12  0.198313           acces
1159     13  0.927388  accessoirement
256       4  0.849370   accompagnemer
...     ...       ...             ...
1664     17  0.973424          visuel
5150     11  0.949526        viticole
710      15  0.931314        vitrerie
441      10  0.978871         voiture
506       9  0.991700          voyage

[1462 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 12, 13, 8, 20, 4, 14, 11, 5, 17, 15, 3, 19, 6, 10, 1, 18, 2, 16])

In [ ]:
from gensim import similarities
index = similarities.MatrixSimilarity(model[corpusBow])

In [ ]:
sims = index[vec_lsi]

In [ ]:
vec_lsi

[(0, 0.02222222),
 (1, 0.02222222),
 (2, 0.02222222),
 (3, 0.02222222),
 (4, 0.02222222),
 (5, 0.02222225),
 (6, 0.02222222),
 (7, 0.02222222),
 (8, 0.02222222),
 (9, 0.02222222),
 (10, 0.02222222),
 (11, 0.02222222),
 (12, 0.02222222),
 (13, 0.3555555),
 (14, 0.3555556)]

In [ ]:
sorted(enumerate(sims), key=lambda item: -item[1])

In [ ]:
doc = "horlogerie metal precieux"
vec_bow = id2word.doc2bow(doc.lower().split())
vec_lsi = model[vec_bow]  # convert the query to LSI space
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for i, s in sims[:200]:
    print(i,s, corpus0[i])